In [1]:
import keras
keras.__version__

'3.1.1'

# 新闻分类：多分类问题

本练习你会构建一个网络，将路透社新闻划分为 46 个互斥的主题。因为有多个类别，所以这是多分类（multiclass classification）问题的一个例子。因为每个数据点只能划分到一个类别，所以更具体地说，这是单标签、多分类（single-label, multiclass classification）问题的一个例子。如果每个数据点可以划分到多个类别（主题），那它就是一个多标签、多分类（multilabel, multiclass classification）问题。


## 路透社数据集

这里使用路透社数据集，它包含许多短新闻及其对应的主题，由路透社在 1986 年发布。它是一个简单的、广泛使用的文本分类数据集。它包括 46 个不同的主题：某些主题的样本更多，但训练集中每个主题都有至少 10 个样本。

与 MNIST 类似，路透社数据集也内置为 Keras 的一部分。我们来看一下。


In [2]:
from keras.datasets import reuters

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

参数 num_words=10000 的意思是仅保留训练数据中前 10 000 个最常出现的单词。低频单词将被舍弃。这样得到的向量数据不会太大，便于处理。

我们有 8982 个训练样本和 2246 个测试样本。（这里下载可能会失败几次，不翻墙可以下的）

In [3]:
len(train_data)

8982

In [4]:
len(test_data)

2246

每个样本都是一个整数列表（表示单词索引）。

In [5]:
train_data[10]

[1,
 245,
 273,
 207,
 156,
 53,
 74,
 160,
 26,
 14,
 46,
 296,
 26,
 39,
 74,
 2979,
 3554,
 14,
 46,
 4689,
 4329,
 86,
 61,
 3499,
 4795,
 14,
 61,
 451,
 4329,
 17,
 12]

如果好奇的话，你可以用下列代码将索引解码为单词。

In [6]:
word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
# Note that our indices were offset by 3（注 意，索引减去了 3）
# because 0, 1 and 2 are reserved indices for "padding", "start of sequence", and "unknown".
#（因为 0、1、2 是 为“padding”（ 填 充 ）、“start of sequence”（序列开始）、“unknown”（未知词）分别保留的索引）
decoded_newswire = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[0]])

In [7]:
decoded_newswire

'? ? ? said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3'

由于限定为前 10 000 个最常见的单词，单词索引都不会超过 10 000。

In [8]:
max([max(sequence) for sequence in train_data])

9999

The label associated with an example is an integer between 0 and 45: a topic index.

样本对应的标签是一个 0~45 范围内的整数，即话题索引编号。

In [9]:
train_labels[10]

3

## Preparing the data

We can vectorize the data with the exact same code as in our previous example:
## 准备数据

你不能将整数序列直接输入神经网络。你需要将列表转换为张量。转换方法有以下两种。

a 填充列表，使其具有相同的长度，再将列表转换成形状为 (samples, word_indices) 的整数张量，然后网络第一层使用能处理这种整数张量的层（即 Embedding 层，本书后面会详细介绍）。

b 对列表进行 one-hot 编码，将其转换为 0 和 1 组成的向量。举个例子，序列 [3, 5] 将会 被转换为 10 000 维向量，只有索引为 3 和 5 的元素是 1，其余元素都是 0。然后网络第一层可以用 Dense 层，它能够处理浮点数向量数据。

下面我们采用后一种方法将数据向量化。为了加深理解，你可以手动实现这一方法，如下所示。

In [10]:
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

# Our vectorized training data（将训练数据向量化）
x_train = vectorize_sequences(train_data)
# Our vectorized test data（将测试数据向量化）
x_test = vectorize_sequences(test_data)

将标签向量化有两种方法：你可以将标签列表转换为整数张量，或者使用 one-hot 编码。

one-hot 编码是分类数据广泛使用的一种格式，也叫分类编码（categorical encoding）。6.1 节给出了 one-hot 编码的详细解释。在这个例子中，标签的 one-hot 编码就是将每个标签表示为全零向量，只有标签索引对应的元素为 1。其代码实现如下。


In [11]:
def to_one_hot(labels, dimension=46):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1.
    return results

# Our vectorized training labels（将训练标签向量化）
one_hot_train_labels = to_one_hot(train_labels)
# Our vectorized test labels（将测试标签向量化）
one_hot_test_labels = to_one_hot(test_labels)

注意，Keras 内置方法可以实现这个操作，你在上文的例子中已经见过这种方法。

In [12]:
from tensorflow.python.keras.utils.np_utils import to_categorical

one_hot_train_labels = to_categorical(train_labels)
one_hot_test_labels = to_categorical(test_labels)

本练习可自由选择编码方法。

## 训练、验证和改进网络

接下来针对这一数据集建立网络模型，这个主题分类问题与MNIST分类问题类似，两个例子都是分类问题。但这个问题有一个新的约束条件：输出类别的数量从变为 46 个。输出空间的维度要大得多。

先应用只包含1个隐藏层、32个单元神经网络训练模型，观察效果。

之后应用本次实验介绍的方法调整隐藏层数、神经元数、（逐步指数级提高学习率，画误差曲线，找到误差升高的点）搜索最佳学习率。保存检查点，使用早停，用 TensorBoard 画学习曲线的图。尝试建立一个你能实现的精度最高的模型。


本次实验报告不要求复现学习材料中内容，请在报告中概括描述在Tensorflow Playboard部分的收获和理解，之后完成本练习。

In [13]:
import os  
from tensorflow.keras.models import Sequential  
from tensorflow.keras.layers import Dense  
from tensorflow.keras.optimizers import Adam  
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard  
  
# 定义模型的绝对日志目录  
logs_dir = 'D:\\Downloads\\logs\\'  
  
# 确保日志目录存在  
if not os.path.exists(logs_dir):  
    os.makedirs(logs_dir) 

In [14]:
# 初始化序贯模型  
model = Sequential()  
  
# 添加输入层到隐藏层的连接，使用ReLU激活函数  
model.add(Dense(32, input_dim=10000, activation='relu'))  

D:\Anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
# 添加隐藏层到输出层的连接，使用softmax激活函数，因为这是一个多分类问题  
model.add(Dense(46, activation='softmax'))  
  
# 编译模型，选择优化器、损失函数和评估指标  
optimizer = Adam()  # 使用Adam优化器  
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])  

In [16]:
# 打印模型概述  
model.summary() 

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │       320,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 46)             │         1,518 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321,550 (1.23 MB)

 Trainable params: 321,550 (1.23 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
# 设置回调函数  
checkpointer = ModelCheckpoint(filepath=os.path.join(logs_dir, 'model.keras'), verbose=1, save_best_only=True)  
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)  
tensorboard_callback = TensorBoard(log_dir=logs_dir, histogram_freq=1, write_graph=True, write_images=True)  

In [18]:
# 训练模型  
model.fit(x_train, one_hot_train_labels,  
          batch_size=32,  
          epochs=20,  
          validation_data=(x_test, one_hot_test_labels),  
          callbacks=[checkpointer, early_stopping, tensorboard_callback])  

Epoch 1/20
273/281 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5718 - loss: 2.2813
Epoch 1: val_loss improved from inf to 1.14759, saving model to D:\Downloads\logs\model.keras
281/281 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.5748 - loss: 2.2608 - val_accuracy: 0.7440 - val_loss: 1.1476
Epoch 2/20
276/281 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8284 - loss: 0.8216
Epoch 2: val_loss improved from 1.14759 to 0.95104, saving model to D:\Downloads\logs\model.keras
281/281 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8287 - loss: 0.8203 - val_accuracy: 0.7890 - val_loss: 0.9510
Epoch 3/20
272/281 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9035 - loss: 0.4622
Epoch 3: val_loss improved from 0.95104 to 0.88869, saving model to D:\Downloads\logs\model.keras
281/281 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9035 - loss: 0.4623 - val_accuracy: 0.8001 - val_loss: 0.8887
Epoch 4/20
276/281 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9365 - loss: 0.3105
Epoch 4: val_lo

In [19]:
# 评估模型  
test_loss, test_accuracy = model.evaluate(x_test, one_hot_test_labels)  
print(f'Test accuracy: {test_accuracy:.4f}')  # 使用格式化字符串来显示测试准确率

71/71 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8171 - loss: 0.8446
Test accuracy: 0.8054
